In [2]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

import openmm as mm
import openmm.app as app
import openmm.unit as unit
from mdtraj.reporters import HDF5Reporter
import copy
import nglview as nv
import random

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

joblib is not installed. Falling back to sequential computation.


In [7]:
base_pair_map = {'A':'T','T':'A','G':'C','C':'G','U':'A','D':'G','E':'T','L':'M','M':'L','B':'S','S':'B','Z':'P','P':'Z'}
        
reference_bases = {base: md.load_hdf5(mdna.utils.get_data_file_path(f'./atomic/bases/BDNA_{base}.h5')) for base in base_pair_map.keys()}

bases = list(reference_bases.values())
order = [['A', 'T', 'G', 'C', 'U'], 
         ['B', 'S', 'Z', 'P'], 
         ['E', 'D','L', 'M']]


# Initialize the trajectory with the first base
traj = reference_bases[order[0][0]]

# Spacing parameters (in angstroms, adjust as needed)
horizontal_spacing = 1.2  # Spacing between bases within a row
vertical_spacing = 1.5    # Spacing between rows

# Base positions tracking
y_position = 0  # Start at the top-most row and work downwards

for row in order:
    x_position = 0  # Reset x position for each new row
    for i, base in enumerate(row,1):
        if i == 0 and row == order[0]:
            # Already initialized with the first base
            continue
        # Move base in x and y direction
        reference_bases[base].xyz[0] = reference_bases[base].xyz[0] + np.array([-x_position, y_position, 0])
        # Stack the base to the trajectory
        traj = traj.stack(reference_bases[base])
        # Increment x position for the next base in the row
        x_position += horizontal_spacing
    # Decrement y position for the next row to position it below the current one
    y_position -= vertical_spacing

In [10]:
subtraj = traj.atom_slice(traj.top.select('not element H'))
subtraj.save_pdb('all_bases.pdb')
subtraj.save_hdf5('all_bases.h5')
view = nv.show_mdtraj(subtraj)    
view.clear()
view.add_representation('licorice', selection='all')
view

NGLWidget()